#Classical Shadows using Numpy

# Introduction

This tutorial explains how to compute the [Classical Shadows](https://arxiv.org/abs/2002.08953) of mixed quantum states and then use these shadows to estimate the expectation of observables, all implemented from scratch in Numpy.

Let $\Phi: \ \mathcal{L}(\mathbb{C} ^ {2 ^ n}) \rightarrow \mathcal{L}(\mathbb{C} ^ {2 ^ n})$ be a quantum channel acting on $n$-qubit operators and let $f$ be a probability distribution defined over the set of all unitaries acting on $\mathbb{C} ^ {2 ^ n}$. Define $T_{(f, \Phi)}: \  \mathcal{L}(\mathbb{C} ^ {2 ^ n}) \rightarrow \mathcal{L}(\mathbb{C} ^ {2 ^ n})$ as $T_{(f, \Phi)} (\rho) =  \int \limits_{U} U  ^ {\dagger} \Phi (U \rho U ^ {\dagger}) U  \ \text{d}f(U) $. This process is called twirling the channel $\Phi$ using the distriubtion $f$. Let $\Phi$ be the measurement (in the computational basis) channel and let $f$ be a distribution such that $T_{(f, \Phi)}$ is invertible. Consider a randomized protocol defined as follows: (i) sample an $n$-qubit unitary $U$ according to $f$, (ii) apply $U$ on $\rho$, (iii) measure all qubits of the resulting state, giving the classical $n$-bit string $b$ as output, (iv) compute $\hat{\rho} = T_{(f, \Phi)} ^ {-1} (U ^ {\dagger} |b\rangle \langle b| U)$ classically.

    
$\hat{\rho}$ is called a *classical shadow* of $\rho$. It is a random variable that reproduces the original state in expectation. For any observable $O$, the variance of the random variable $\text{tr}(\hat{\rho}O)$ is upper bounded by a quantity called the *shadow norm* of $O$, denoted as $||O||_{\text{sh}}$, which is independent of the state $\rho$ but dependent on $f$, and hence we have the following theorem


  **Theorem**: Let $\rho \in \mathbb{C} ^ {2 ^ n}$ be a quantum state, $O_1, O_2, \dots, O_m \in \mathbb{C} ^ {2 ^ n}$ be Hermitian matrices and $\epsilon, \delta \in \left[ 0, 1\right]$ be accuracy parameters. By using $ T = \mathcal{O} (\frac{\log{(M/ \delta)}}{\epsilon ^ 2} \max \limits_i ||O_i||_{\text{sh}})$ classical shadows $\left\{ \hat{\rho}_1, \hat{\rho}_2, \dots, \hat{\rho}_T \right\}$, we have $\left| \overline{o}_i - \text{tr} (\rho O_i) \right| \leq \epsilon \ \forall \ i$ with probability at least $1 - \delta$. Here, $\overline{o}_i$ is a median of means estimator of samples $\text{tr}\left(\hat{\rho}_1 O_i\right), \text{tr}\left( \hat{\rho}_2 O_i\right), \dots, \text{tr}\left( \hat{\rho}_T O_i\right)$.

  This is remarkable because the number of shadows required to estimate the expectation of the observables depends only on (i) the shadow norm of the observable, (ii) the precision to which we want to estimate the expectation, and (iii) the error probability, and not on the state itself. Also, to estimate the expectation of $M$ observables, we only require $\mathcal{O}(\log M)$ classical shadows loaded. To make this procedure efficient, we should have $f$ defined over unitaries that enables computing $T_{(f, \Phi)} ^ {-1} (U ^ {\dagger} | b \rangle \langle b |U) $ classically feasible, with classical cost polynomial in $n$. Two such distributions $f$ have been proposed, each resulting in a different range of values for the shadow norm



*   *Clifford ensemble*: Here, $f$ is the discrete uniform distribution over all $n$-qubit Clifford unitaries. In this case, $ T_{(f, \Phi)} ^ {-1}(\rho) = \left(2 ^ n + 1\right) \rho - \mathbf I$. The stabilizer formalism can be used to work with classical shdows in this scenario. In this case, $|| O||_{\text{sh}} = \mathcal{O}(\text{tr}(O ^ 2))$.
*   *Pauli ensemble*: Here, $f$ is the discrete uniform distribution over the set $\left\{ \mathbf I, H, HS ^ {\dagger}\right\} ^ {\otimes n}$. In this case, $ T_{(f, \Phi)} ^ {-1}(\rho) = \bigotimes \limits_{i = 1} ^ n \left( 3 \rho - \mathbf I\right)$ and $|| O||_{\text{sh}} = \mathcal{O}(4 ^ k)$, where $k$ is the total number of qubits which is non-trivially acted upon by $O$





    

  For example, if the observable is a quantum state (fidelity with a pure state), one can choose to use the Clifford ensemble, and if the observable is a local observable (or can be written as a linear combination of a few local observables), then we can use the Pauli ensemble. Many relevant observables in nature can be written as linear combinations of a few observables that act non-trivially on only a few qubits. This makes classical shadows a very powerful ``storage facility'' for states as well. In many contexts, this is why classical shadows are considered an alternative to conventional quantum state tomography, which requires resources exponential in the number of qubits. The key fact here is that to estimate many properties of quantum states, up to acceptable precision and quality, one does not need to know the full classical description of the states. Similar complexity guarantees are extended to other important problems involving quantum states such as finding entanglement witnesses, direct fidelity estimation, estimating the output of non-linear functions, etc. 

In this tutorial we shall load classical shadows of quantum states using the Clifford ensemble as well as the Pauli ensemble. Using the Pauli ensemble, we will estimate the expectation of local Pauli observables and with the Clifford ensemble, we will estimate the fidelity with a target pure stabilizer state.

# Importing packages

In [1]:
import time
import numpy as np
import math
import random

# Required functions

## Matrix initialisations and basic functions

### Gates

In [3]:
H_gate = (0.5 ** (0.5)) * np.matrix([[1,1], 
               [1,-1]])

X_gate = np.matrix([[0 + 0j, 1 + 0j], 
               [1 + 0j, 0+ 0j]])

Y_gate = np.matrix([[0 + 0j, -1j], 
               [1j, 0 + 0j]])

Z_gate = np.matrix([[1 + 0j, 0 + 0j], 
               [0 + 0j, -1 + 0j]])

Id = np.matrix([[1 + 0j, 0 + 0j], 
                [0 + 0j, 1 + 0j]])


Y_basis = (1/(2 ** 0.5)) * np.matrix([[1 + 0j, -1j], 
                                      [1 + 0j, 1j]]) # This is the gate HS ^ \dagger

Y_basis_dag = np.conj(Y_basis).T


CNOT = np.matrix([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j], 
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j]])

S_gate = np.matrix([[1 + 0j, 0 + 0j], 
                    [0 + 0j, 0 + 1j]])

S_dag_gate = np.matrix([[1 + 0j, 0 + 0j], 
                    [0 + 0j, 0 - 1j]])

clifford_gate_dict = {'H': H_gate, 'S': S_gate, 'X':X_gate, 'Y': Y_gate, 'Z':Z_gate, 
                      'CX': CNOT, 'Y_basis': Y_basis, 'Y_basis_dag': Y_basis_dag, 'I': Id, 'S_dag': S_dag_gate}

### Tables

For each of the Clifford gates, we shall store their actions in tables so that this can be used to update the stabilizer tables of the stabilizer states we will be working with. More information regarding stabilizer tables and updating them by action of Clifford gates, as well as measuring them, can be seen in [Nielsen and Chuang](http://mmrc.amss.cas.cn/tlb/201702/W020170224608149940643.pdf) Stabilizer codes section. We will be following the techniques explained there.

As an example, consider the Hadamard gate. This gate maps $\mathbf{I}$ to itself, $X$ to $Z$, $Y$ to $-Y$ and $Z$ to $X$. By labelling $\mathbf{I}$ as index $0$, $X$ as index $1$, $Y$ as index $2$ and $Z$ as index $3$, we store the action of $H$ gate on Paulis using the martix "H_table" given in the next cell. The first row is $[0,1]$, meaning that $\mathbf{I}$ is mapped to $\mathbf{I}$ and a sign of $+1$ is multiplied. Similarly, all the actions discussed above can be seen in the table. For all the one qubit Clifford gates, we have stored their actions in similar tables.

When it comes to the CNOT gate, we need to store its action on all possible pairs of Paulis. Hence we store its actions in a dictionary with $2$-tuples as keys and $3$-tuples as values. This dictionary can also be read in a similar way to the tables of the one-qubit gates. For example: $(1, 3): (2, 2, -1)$ means CNOT maps $X \otimes Z$ to $-Y \otimes Y$.

We also have a multiplcation table "pauli_multi_table" which can be read as follows: pauli_multi_table$[i][j]$ = $(ij, \text{induced phase})$. Here, $i$ and $j$ refers to the indices that we have given to the Paulis, and the "induced phase" is the phase that could come as a result of multiplication between Paulis. For example, $XZ$ gives us $Y$ with a phase of $-i$. 

An $n$-qubit pure state stabilized by a generator set $\langle P_1, P_2, \dots, P_n \rangle$ is stored using a $n \times (n + 1)$ matrix, with its $i ^ {th}$ row denoting the Pauli string $P_i$, stored as a row of numbers each belonging in $\left\{ 0, 1, 2, 3\right\}$ (similar indexing as previously discussed). The last column stores the sign of the Pauli, that is, one of $\left\{ 1, -1, i, -i\right\}$. 

In [8]:

H_table = np.array([[0, 1], 
                    [3, 1], 
                    [2, -1],
                    [1, 1]], dtype = 'complex128')

S_table = np.array([[0, 1], 
                    [2, 1], 
                    [1, -1],
                    [3, 1]], dtype = 'complex128')

S_dag_table = np.array([[0, 1], 
                        [2, -1], 
                        [1, 1],
                        [3, 1]], dtype = 'complex128')

X_table = np.array([[0, 1], 
                    [1, 1], 
                    [2, -1],
                    [3, -1]], dtype = 'complex128')

Y_table = np.array([[0, 1], 
                    [1, -1], 
                    [2, 1],
                    [3, -1]], dtype = 'complex128')

Z_table = np.array([[0, 1], 
                    [1, -1], 
                    [2, -1],
                    [3, 1]], dtype = 'complex128')


pauli_multi_table = [[(0 + 0j, 1 + 0j), (1 + 0j, 1 + 0j), (2 + 0j, 1 + 0j), (3 + 0j, 1 + 0j)],
                    [(1 + 0j, 1 + 0j), (0 + 0j, 1 + 0j), (3 + 0j, 0 + 1j), (2 + 0j, 0 - 1j)],
                    [(2 + 0j, 1 + 0j), (3 + 0j, 0 - 1j), (0 + 0j, 1 + 0j), (1 + 0j, 0 + 1j)],
                    [(3 + 0j, 1 + 0j), (2 + 0j, 0 + 1j), (1 + 0j, 0 - 1j), (0 + 0j, 1 + 0j)]]

cnot_table = {(0 + 0j, 0 + 0j): (0 + 0j, 0 + 0j, 1 + 0j),
              (0 + 0j, 1 + 0j): (0 + 0j, 1 + 0j, 1 + 0j),
              (0 + 0j, 2 + 0j): (3 + 0j, 2 + 0j, 1 + 0j),
              (0 + 0j, 3 + 0j): (3 + 0j, 3 + 0j, 1 + 0j),
              (1 + 0j, 0 + 0j): (1 + 0j, 1 + 0j, 1 + 0j),
              (1 + 0j, 1 + 0j): (1 + 0j, 0 + 0j, 1 + 0j),
              (1 + 0j, 2 + 0j): (2 + 0j ,3 + 0j, 1 + 0j),
              (1 + 0j, 3 + 0j): (2 + 0j ,2 + 0j, -1 + 0j),
              (2 + 0j, 0 + 0j): (2 + 0j, 1 + 0j, 1 + 0j),
              (2 + 0j, 1 + 0j): (2 + 0j, 0 + 0j, 1 + 0j),
              (2 + 0j, 2 + 0j): (1 + 0j, 3 + 0j, -1 + 0j),
              (2 + 0j, 3 + 0j): (1 + 0j, 2 + 0j, 1 + 0j),
              (3 + 0j, 0 + 0j): (3 + 0j, 0 + 0j, 1 + 0j),
              (3 + 0j, 1 + 0j): (3 + 0j, 1 + 0j, 1 + 0j),
              (3 + 0j, 2 + 0j): (0 + 0j, 2 + 0j, 1 + 0j),
              (3 + 0j, 3 + 0j): (0 + 0j, 3 + 0j, 1 + 0j)}




clifford_table_dict = {'X': X_table, 'Y': Y_table, 'Z': Z_table, 'S':S_table, 'S_dag':S_dag_table, 'H':H_table}


# This function is used to convert an integer to a list of binary values.
# 1. x: integer value
# 2. BitNo: how many binary bits for x's expansion is required
def binary(x, BitNo):
  format(x, 'b').zfill(BitNo)
  Binlist = [int(y) for y in list(format(x, 'b').zfill(BitNo))]
  return Binlist


## Tensor operations

In [5]:
# This function applies a gate on specific qubits of the state and returns the resulting state using matricisation of the tensor. 

# 1. state: Numpy array (vector)
# 2. qubits_to_act: list of integers (0 is for the first qubit)
# 3. gate: numpy matrix


def across_qubits_gate(state, qubits_to_act, gate):
  NQubits = int(np.log2(len(state)))

  rest_of_qubits = []
  for i in range(NQubits):
    if i not in qubits_to_act:
      rest_of_qubits.append(i)

  state_tensor = np.reshape(state, tuple([2] * NQubits))
  matricised_tensor = np.reshape(np.ndarray.transpose(state_tensor, qubits_to_act + rest_of_qubits), (2 ** len(qubits_to_act), 2 ** len(rest_of_qubits)))
  answer = np.matmul(gate, matricised_tensor)
  answer = np.asarray(answer)
  p = qubits_to_act + rest_of_qubits

  s = np.zeros(len(p), dtype = int)
  
  for i in list(range(len(p))):
      s[p[i]] = i

  Z = np.ndarray.transpose(np.reshape(answer, tuple([2] * NQubits)), s)
  Z = np.reshape(Z, int(2 ** NQubits))

  return(Z)


# Pauli Ensemble

In this section, we shall use the Pauli ensemble to generate Classical shadows and use it to estimate expectation with $1$ and $2$ local Pauli strings. The procedure is to simply measure each qubit in a uniformly randomly sampled Pauli basis, and then apply (classically) the function $T(X) = 3X-\mathbf I$ on each qubit separately. So, that means that each shadow will be a fully separable matrix and hence, for an $n$-qubit state, a single shadow can be stored using a $4n$ dimensional vector.

Let $\rho \in \mathcal{L}(\mathbb{C} ^ {2^n})$ be an $n$-qubit quantum state and let $\left\{ \rho_i = \bigotimes \limits_{j = 1} ^ n \rho_i ^ {(j)} \Big| \ i = 1, 2, \dots, T\right\}$ be its $T$ classical shadows loaded using the Pauli ensemble. This set can be stored using a $T \times 4n$ dimensional block matrix as follows:

\begin{align*}
  \begin{bmatrix}
    \text{vec}(\rho_1 ^ {(1)}) ^ T & \text{vec}(\rho_1 ^ {(2)}) ^ T & \dots & \text{vec}(\rho_1 ^ {(n)}) ^ T \\
    \text{vec}(\rho_2 ^ {(1)}) ^ T & \text{vec}(\rho_2 ^ {(2)}) ^ T & \dots & \text{vec}(\rho_2 ^ {(n)}) ^ T \\
    \vdots \\
    \text{vec}(\rho_T ^ {(1)}) ^ T & \text{vec}(\rho_T ^ {(2)}) ^ T & \dots & \text{vec}(\rho_T ^ {(n)}) ^ T
  \end{bmatrix}
\end{align*}

To estimate the expectation of a Pauli string, with a single non-identity term $P$ featuring in the $i^{\text{th}}$ qubit, we simply multiply the $i^{th}$ block column of this matrix with the vector $\overline{\text{vec}(P)}$. This results in a $T \times 1$ column vector and a median of means estimator of the entries of the vector gives us the estimate of the expectaion value. The reason that we choose this way is that for any two matrices $X$ and $Y$, to compute the value $\text{tr}(X^{\dagger} Y)$, it is inefficient to compute the matrix $X ^ {\dagger} Y$. $\text{vec}(X) ^ {\dagger} \text{vec}(Y)$ gives the same answer with much less floating point computations. Even though we are computing the conjugate of this value, since the expectation is a real value, it will not change anything.

If the Pauli string has multiple non-identity terms, simply compute the $T \times 1$ column vector associated with each term, and then compute an element-wise product of the column vectors. Then, a median of means estimator results in the estimate of the expectation value.

In [45]:
# This function computes T Pauli Classical Shadows of a mixed state and returns it in the matrix form presented above.
# 1. mixture: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. T: Number of Classical shadows to be computed

def pcs(mixture, T):
  num_qubits = int(np.log2(len(mixture[0][1])))
  shadow_list = []
  gate_set_list = ['H', 'Y_basis']
  
  for j in range(T):
    temp_mixture = mixture.copy()
    final_prob_array = np.zeros(2 ** num_qubits)
    gate_list = []
    for i in range(num_qubits):
      temp = np.random.choice(3, 1, p=[1/3, 1/3, 1/3])[0]
      if temp != 2:
        gate = gate_set_list[temp]
        gate_list.append((gate, i))
        for k in range(len(temp_mixture)):
          temp_state = across_qubits_gate(temp_mixture[k][1], [i], clifford_gate_dict[gate])
          temp_mixture[k] = (temp_mixture[k][0], temp_state)
      else:
        gate_list.append(('I', i))
    for pair in temp_mixture:
      prob_array = np.array([np.abs(pair[1][k]) ** 2 for k in range(len(pair[1]))])
      final_prob_array += pair[0] * prob_array
    outcome = np.random.choice(2 ** num_qubits, 1, p = list(final_prob_array))[0]
    outcome_bin = binary(outcome, num_qubits)
    final_outcome = np.array([])
    for i in range(len(gate_list)):
      if gate_list[i][0] == 'Y_basis':
        temp1 = clifford_gate_dict['Y_basis_dag']
      else:
        temp1 = clifford_gate_dict[gate_list[i][0]]
      temp = temp1[:, outcome_bin[i]]
      final_outcome = np.concatenate([final_outcome, np.array((3 * np.outer(temp, np.conj(temp))) - Id).flatten()], axis = 0)

    shadow_list.append(final_outcome)
  shadow_list = np.vstack(shadow_list)
  return shadow_list

# This function returns the expectaion value of a Pauli observable using the method explained above.
# 1. shadow_matrix: Shadows as a matrix. Output of the previous function.
# 2. observable_list: A list of single qubit Paulis and the indices on which they are acting. For example [(0, 'X'), (3, 'Z')] denotes the Pauli string X I I Z.
# 3. N: Parameter in median of means estimation
# 4. K: Parameter in median of means estimation
def pcs_expectation(shadow_matrix, observable_list, N, K):
  temp_mat = np.zeros((shadow_matrix.shape[0], len(observable_list)), dtype = 'complex128')
  k = 0
  for pair in observable_list:
    temp = shadow_matrix[:, pair[0] * 4: (pair[0] + 1) * 4]
    flattened_observable = np.conj(np.array(clifford_gate_dict[pair[1]].flatten())[0])
    temp_mat[:, k] = temp @ flattened_observable
    k += 1

  exp_mat = np.prod(temp_mat, axis = 1)
  chunks = [np.mean(exp_mat[x:x+N]) for x in range(0, len(exp_mat), K)]
  return np.median(chunks)



Now, let's prepare a mixture of $| + \rangle ^ {\otimes 4}$ with $0.8$ probability and a Haar random state with $0.2$ probability.

In [68]:

n_qubits = 4 # Number of qubits the state should be prepared on 

global_mixture = [] # The mixture is stored in this list

state = np.ones(2 ** n_qubits) # Plus state
state = state/np.linalg.norm(state)

global_mixture.append((0.8, state))

state = np.random.uniform(-1, 1, 2 ** n_qubits) + (1j * np.random.uniform(-1, 1, 2 ** n_qubits)) # Haar random state
state = state/np.linalg.norm(state)

global_mixture.append((0.2, state))



Let the local Pauli string be $X I I I$. So the acual expectation value of our mixture with this Pauli string is

In [69]:
exp_val = 0
for pair in global_mixture:
  temp = across_qubits_gate(pair[1], [0], X_gate)
  exp_val += pair[0] * np.dot(np.conj(pair[1]), temp)

print("Actual expectation value: {}".format(exp_val))

Actual expectation value: (0.8792299857815542+5.551115123125783e-18j)


Now, let's estimate this value using $4000$ Classical shadows.

In [70]:
shadows = pcs(global_mixture, 4000)

In [71]:
pcs_expectation(shadows, [(0, 'X')], N = 50, K = 80)

(0.8400000000000003+0j)

As we can see, the estimated value is close to the true value.

Now we shall choose a different observable, the Pauli string $X I Y I$. Then, the actual expectation value is given as 

In [72]:
exp_val = 0
for pair in global_mixture:
  temp = across_qubits_gate(pair[1], [0], X_gate)
  temp = across_qubits_gate(temp, [2], Y_gate)
  exp_val += pair[0] * np.dot(np.conj(pair[1]), temp)

print("Actual expectation value: {}".format(exp_val))

Actual expectation value: (0.013211411954024485-3.4694469519536144e-19j)


Using Classical shadows, we get the estimate

In [73]:
pcs_expectation(shadows, [(0, 'X'), (2, 'Y')], N = 50, K = 80)

0j

This is also pretty close to the actual answer.

# Clifford Ensemble

In this section, we shall use the Classical shadows generated using the Clifford ensemble to estimate the fidelity between a mixed state $\rho$ and a pure stabilizer state $| \psi \rangle$, that is, the value $\langle \psi | \rho | \psi \rangle $. Let $\left\{ \rho_i = (2 ^ n + 1) U_i ^ {\dagger} | b_i \rangle \langle b_i | U_i - \mathbf{I}\ | \ i = 1,2, \dots, T \right\}$ be shadows of $\rho$ generated using the Clifford ensemble. Then, each $\rho_i$ is stored as a stabilizer table. The fidelity between $|\psi\rangle$ is then estimated using the median of means estimator of the values $\left\{ (2 ^ n + 1)\left| \langle \psi | U_i ^ {\dagger} | b_i \rangle \right| ^ 2 - 1 \ \Big| \ i = 1, 2, \dots, T \right\}$. So, we should be able to compute these fidelity values efficiently.

On top of $| \psi \rangle$ being a stabilizer state, we also assume that we know the Clifford circuit that can be used to prepare $| \psi \rangle$. This will allow us to compute the fidelity values efficiently. To compute the pure state fidelity between two stabilizer states $|\alpha \rangle = U |0 \rangle$ and $|\beta \rangle = V |0 \rangle$, first we compute the stabilizer table of the state given as $V ^ {\dagger} U | 0 \rangle$. Then, by using property of stabilizer formalism, we simply compute the probability that all qubits will be measured to be in $0$. This gives us the required fidelity. 

Such a measurement will require us to check whether a given Pauli string is an element of the subgroup generated by the Paulis in the stabilizer table. This is implemented using the Row-Reduced Echelon Form technique discussed in this [work](https://studentutsedu-my.sharepoint.com/personal/afrad_m_basheer_student_uts_edu_au/Documents/Microsoft%20Teams%20Chat%20Files/Dimitris-Kakofengitis-Dissertation.pdf). To generate a circuit uniformly sampled from the Clifford group, we shall use the technique given in this [work](https://arxiv.org/abs/2008.06011).

In [44]:
# This function generates a circuit which is uniformly sampled from the Clifford group.
# 1. n_qubits: Number of qubits the circuit should act on

def generate_random_clifford_circuit(n_qubits):

  gate_list = []

  for i in range(n_qubits):
    m = n_qubits - i
    while 1 == 1:
      
      a_x = np.random.choice(2, m, p = [0.5, 0.5])
      b_x = np.random.choice(2, m, p = [0.5, 0.5])

      a_z = np.random.choice(2, m, p = [0.5, 0.5])
      b_z = np.random.choice(2, m, p = [0.5, 0.5])
      count = 0
      for j in range(m):
        if (a_x[j], a_z[j]) != (b_x[j], b_z[j]):
          if (a_x[j], a_z[j]) != (0, 0) and (b_x[j], b_z[j]) != (0, 0):
            count += 1
      if (count % 2) == 1:
        break
    x = np.vstack([a_x, b_x]) 
    z = np.vstack([a_z, b_z])
    
    for j in range(m):
      if z[0, j] == 1:
        if x[0, j] == 0:
          gate_list.append(('H', [j + i]))
          temp = x[:, j].copy()
          x[:, j] = z[:, j]
          z[:, j] = temp
        else:
          gate_list.append(('S', [j + i]))
          z[:, j] += x[:, j]
          z = z % 2

    j_list = []
    for j in range(m):
      if x[0, j] == 1:
        j_list.append(j)

    flag = 0
    while flag == 0:
      m_1 = len(j_list)
      new_j_list = []
      for j in range(0, m_1, 2):
        if j != m_1 - 1:
          gate_list.append(('CX', [j_list[j] + i, j_list[j+1] + i]))
          z[:, j_list[j]] += z[:, j_list[j + 1]]
          z = z % 2

          x[:, j_list[j + 1]] += x[:, j_list[j]]
          x = x % 2
        new_j_list.append(j_list[j])

      j_list = new_j_list
      if len(j_list) == 1:
        flag += 1

    if j_list[0] != 0:
      gate_list.append(('CX', [0 + i, j_list[0] + i]))
      z[:, 0] += z[:, j_list[0]]
      z = z % 2

      x[:, j_list[0]] += x[:, 0]
      x = x % 2

      gate_list.append(('CX', [j_list[0] + i, 0 + i]))
      z[:, j_list[0]] += z[:, 0]
      z = z % 2

      x[:, 0] += x[:, j_list[0]]
      x = x % 2

      gate_list.append(('CX', [0 + i, j_list[0] + i]))
      z[:, 0] += z[:, j_list[0]]
      z = z % 2

      x[:, j_list[0]] += x[:, 0]
      x = x % 2
    flag1 = 0
    for j in range(1, m):
      if z[1, j] == 1:
        flag1 += 1

    flag1 += ((1 + z[1, 0]) % 2)
    flag1 += sum(x[1,:])

    if flag1 != 0:

      gate_list.append(('H', [0 + i]))
      temp = x[:, 0].copy()
      x[:, 0] = z[:, 0]
      z[:, 0] = temp

      for j in range(m):
        if z[1, j] == 1:
          if x[1, j] == 0:
            gate_list.append(('H', [j + i]))
            #x[:, j], z[:, j] = z[:, j], x[:, j]
            temp = x[:, j].copy()
            x[:, j] = z[:, j]
            z[:, j] = temp
          else:
            gate_list.append(('S', [j + i]))
            z[:, j] += x[:, j]
            z = z % 2

      j_list = []
      for j in range(m):
        if x[1, j] == 1:
          j_list.append(j)
      flag = 0
      m_1 = 0

      while flag == 0:
        m_1 = len(j_list)
        new_j_list = []
        for j in range(0, m_1, 2):
          if j != m_1 - 1:
            gate_list.append(('CX', [j_list[j] + i, j_list[j+1] + i]))
            z[:, j_list[j]] += z[:, j_list[j + 1]]
            z = z % 2

            x[:, j_list[j + 1]] += x[:, j_list[j]]
            x = x % 2
          new_j_list.append(j_list[j])

        j_list = new_j_list

        if len(j_list) == 1:
          flag += 1

      gate_list.append(('H', [0 + i]))
      temp = x[:, 0].copy()
      x[:, 0] = z[:, 0]
      z[:, 0] = temp
    a_s = np.random.choice(2, 1, p = [0.5, 0.5])[0]
    b_s = np.random.choice(2, 1, p = [0.5, 0.5])[0]

    if a_s == 0:
      if b_s == 1:
        gate_list.append(('X', [0 + i]))

    else:
      if b_s == 0:
        gate_list.append(('Z', [0 + i]))
      else:
        gate_list.append(('Y', [0 + i]))

  return gate_list

# This function converts a binary string (standard basis vector) into its corresponding stabilizer table.
# 1. bin_list: List of binary values
def convert_bit_string(bin_list):
  n_qubits = len(bin_list)
  answer = np.zeros((n_qubits, n_qubits + 1), dtype = 'complex128')
  for i in range(n_qubits):
    if bin_list[i] == 0:
      answer[i][i] = 3 + 0j
      answer[i][n_qubits] = 1 + 0j
    else:
      answer[i][i] = 3 + 0j
      answer[i][n_qubits] = -1 + 0j
  return answer

# This function applies a Clifford gate on a stabilizer state given as a stabilizer table, and returns the updated table.
# 1. stab: Stabilizer table of the state. n x n+1 numpy matrix
# 2. gate: The Clifford gate to act given as a letter. For example 'H' for Hadamard
# 3. qubits_to_act: The qubit on which the gate should be applied. Note that the first qubit is 0 and the qubits should always be given in a list. 
def apply_gate_stabilizer(stab, gate, qubits_to_act):
  n_qubits = stab.shape[0]
  if len(qubits_to_act) == 1:
    for i in range(n_qubits):
      stab[i][qubits_to_act[0]] = clifford_table_dict[gate][int(np.real(stab[i][qubits_to_act[0]]))][0]
      
      stab[i][n_qubits] = stab[i][n_qubits] * clifford_table_dict[gate][int(np.real(stab[i][qubits_to_act[0]]))][1]
  else:
    for i in range(n_qubits):
      temp = cnot_table[(stab[i][qubits_to_act[0]], stab[i][qubits_to_act[1]])]
      stab[i][qubits_to_act[0]] = temp[0]
      stab[i][qubits_to_act[1]] = temp[1]
      stab[i][n_qubits] = stab[i][n_qubits] * temp[2]

  return stab

# This function checks if a given Pauli string commutes with element in the generator set (rows of the stabilizer table). 
# It returns (does it commute with everyone, updated stabilizer table with only one generator anti commuting with the pauli string, the index of the generator that it anti-commutes with)
# 1. stab: Stabilizer table of th state. n x n+1 numpy matrix
# 2. pauli_string: The Pauli string given as a list of previously mentioned indices for Paulis
def commutator_check(stab, pauli_string):
  n_qubits = len(pauli_string)
  for i in range(n_qubits):
    sign = 1
    for j in range(n_qubits):
      if stab[i][j] != pauli_string[j] and (stab[i][j] * pauli_string[j]) != 0:
        sign *= -1

    if sign != 1:
      
      for k in range(i + 1, n_qubits):
        sign = 1
        for l in range(n_qubits):
          if stab[k][l] != pauli_string[l] and (stab[k][l] * pauli_string[l]) != 0:
            sign *= -1

        if sign != 1:
          stab = row_addition(stab, [i, k])
      return 0, i, stab

  return 1, 1000, stab

# This function can be used to multiply a specific generator on another generator. It returns the updated stabilizer table.
# 1. stab: Stabilizer table of the state. n x n+1 numpy matrix
# 2. rows: If P_0 is row 0 and P_2 is row 2, to make row 2 P_1P_2, give this argument [0, 2]
def row_addition(stab, rows):
  n_qubits = stab.shape[1] - 1
  for i in range(n_qubits):
    stab[rows[1]][i] = pauli_multi_table[int(np.real(stab[rows[0]][i]))][int(np.real(stab[rows[1]][i]))][0]
    stab[rows[1]][n_qubits] *= pauli_multi_table[int(np.real(stab[rows[0]][i]))][int(np.real(stab[rows[1]][i]))][1]

  stab[rows[1]][n_qubits] *= stab[rows[0]][n_qubits]
  return stab

# The rref subroutine. The returns the rref form of the stabilizer table.
# 1. stab: Stabilizer table of the state. n x n+1 numpy matrix
def rref(stab):
  n_qubits = stab.shape[1] - 1
  number_of_generators = stab.shape[0]
  current_row = 0
  for column in range(n_qubits):
    uni = np.unique(stab[current_row:, column], return_counts=True)[0]
    uni_non_zero = [x for x in uni if x != 0]
    if len(uni_non_zero) == 1:
      ks = []
      for row in range(current_row, number_of_generators):
        if stab[row][column] != 0:
          ks.append(row)

      stab[[ks[0], current_row]] = stab[[current_row, ks[0]]]
      for row in range(current_row + 1, number_of_generators):
        if stab[row][column] != 0:
          stab = row_addition(stab, [current_row, row])
      current_row += 1
      
    elif len(uni_non_zero) >= 2:
      ks = []
      for row in range(current_row, number_of_generators):
        
        if stab[row][column] != 0:
          if len(ks) < 1:
            ks.append((stab[row][column], row))

          elif len(ks) < 2:
            if ks[0][0] != stab[row][column]:
              ks.append((stab[row][column], row))

      stab[[ks[0][1], current_row]] = stab[[current_row, ks[0][1]]]
      stab[[ks[1][1], current_row + 1]] = stab[[current_row + 1, ks[1][1]]]

      if current_row + 2 <= number_of_generators-1:
        for row in range(current_row + 2, number_of_generators):
          if stab[row][column] == ks[0][0]:
            stab = row_addition(stab, [current_row, row])
          elif stab[row][column] == ks[1][0]:
            stab = row_addition(stab, [current_row + 1, row])
          else:
            if stab[row][column] != 0:
              stab = row_addition(stab, [current_row, row])
              stab = row_addition(stab, [current_row + 1, row])

      current_row += 2
  return stab


# This function computes T classical shadows of a state using the Clifford ensemble. It returns a list of T stabilizer tables.
# 1. mixture: A list of 2-tuples of the form [(p_1, \ket{\psi_1}), (p_2, \ket{\psi_2}), ... ,(p_t, \ket{\psi_t})]. Here, p_i are probabilities and \ket{\psi_i} are pure states in the mixture.
# 2. T: number of shadows to load
def ccs(mixture, T):
  num_qubits = int(np.log2(len(mixture[0][1])))
  dim = 2 ** num_qubits
  stab_list = []
  for j in range(T):
    outer_prob_array = np.zeros(dim)
    randomly_generated_clifford_circuit = generate_random_clifford_circuit(num_qubits)
    m = len(randomly_generated_clifford_circuit)
    for pair in mixture:
      Ustate = np.copy(pair[1])
      for i in range(m):
        Ustate = across_qubits_gate(Ustate, randomly_generated_clifford_circuit[i][1], clifford_gate_dict[randomly_generated_clifford_circuit[i][0]])
      outer_prob_array += pair[0] * np.array([np.abs(Ustate[k]) ** 2 for k in range(dim)])
    outcome = np.random.choice(dim, 1, p = list(outer_prob_array))[0]
    outcome_bin = binary(outcome, num_qubits)
    stab = convert_bit_string(outcome_bin)
    randomly_generated_clifford_circuit.reverse()
    for i in range(m):
      if randomly_generated_clifford_circuit[i][0] == 'S':
        stab = apply_gate_stabilizer(stab, 'S_dag', randomly_generated_clifford_circuit[i][1])
      else:
        stab = apply_gate_stabilizer(stab, randomly_generated_clifford_circuit[i][0], randomly_generated_clifford_circuit[i][1])
    stab_list.append(stab)
  return stab_list

# This function returns the probability that the given stabilizer state will result in all zeros after measrement of all qubits.
# 1. stab: Stabilizer table of the state. n x n+1 numpy matrix
def measure_clifford_prob_0(stab):
  prob_list = []
  for qubit in range(stab.shape[0]):
    pauli_string = list(3 * np.eye(stab.shape[0], dtype = 'complex128')[qubit])
    temp, index, stab = commutator_check(stab, pauli_string)
    
    if temp == 1:
      
      updated_pauli_string = np.concatenate([pauli_string, np.array([1 + 0j])], axis = 0)
      stab1 = np.concatenate([stab, np.array([updated_pauli_string])], axis = 0)
      
      stab1 = rref(stab1)
      flag = 0
      for i in range(stab1.shape[0]):
        if np.linalg.norm(stab1[i,:-1]) == 0 and stab1[i,-1] == 1:
          flag = 1
          
      if flag == 1:
        prob_list.append(1)
      else:
        prob_list.append(0)

    else:
      prob_list.append(0.5)
      stab[index] = np.concatenate([np.array(pauli_string), np.array([1 + 0j])], axis = 0)

  return np.prod(np.array(prob_list))

# This function computes the fidelity between two pure stabilizer states, one given as a stabilizer table, and the other given as a circuit that prepares the state.
# 1. stab: Stabilizer table of the state. n x n+1 numpy matrix
# 2. circuit: The circuit that prepares the other state. For example: [('H', [0]), ('CNOT', [1,5]), ('S', [3])]
def compute_fidelity_with_circuit(stab, circuit):
  for i in range(len(circuit)-1, -1, -1):
    if circuit[i][0] == 'S':
      stab = apply_gate_stabilizer(stab, 'S_dag', circuit[i][1])
    else:
      stab = apply_gate_stabilizer(stab, circuit[i][0], circuit[i][1])

  return measure_clifford_prob_0(stab)

# This function estimates the fidelity of a state given as its shadows with another stabilizer state whose preparation circuit is given
# 1. shadows: A list of stabilizer tables. Output of the ccs function.
# 2. circuit: The circuit that prepares the other state. For example: [('H', [0]), ('CNOT', [1,5]), ('S', [3])]
# 3. N: Parameter in median of means estimation
# 4. K: Parameter in median of means estimation
def compute_fidelity_with_shadows(shadows, circuit, N, K):
  exp_list = []
  num_qubits = shadows[0].shape[0]
  for stab in shadows:
    temp = compute_fidelity_with_circuit(stab, circuit)
    exp_list.append((((2 ** num_qubits) + 1) * temp) - 1)
  chunks = [np.mean(exp_list[x:x+N]) for x in range(0, len(exp_list), K)]
  return np.median(chunks)

Now, let's prepare a mixture of a randomly generated pure stabilizer state $| \psi \rangle$ with probability $0.8$ and a Haar random state with probability $0.2$. We shall try to estimate the fidelity between this mixture and $| \psi \rangle$.

In [55]:
n_qubtis = 4
cliff_state = np.array([1] + [0] * ((2 ** n_qubtis)-1)) # initial state ket 0
circ = generate_random_clifford_circuit(4) # randomly generated clifford circuit
for pair in circ:
  cliff_state = across_qubits_gate(cliff_state, pair[1], clifford_gate_dict[pair[0]])

haar_random_state = np.random.uniform(-1, 1, 2 ** n_qubits)
haar_random_state = haar_random_state/np.linalg.norm(haar_random_state)

global_mixture = [(0.8, cliff_state), (0.2, haar_random_state)]

The actual fidelity value is given as

In [56]:
fid_val = 0
for pair in global_mixture:
  fid_val += pair[0] * np.abs(np.dot(np.conj(pair[1]), cliff_state)) ** 2

print("Actual Fidelity: {}".format(fid_val))

Actual Fidelity: 0.8020757052299634


Now, we shall use Classical shadows to estimate this quantity.

In [57]:
shadows = ccs(global_mixture, 4000)

In [58]:
compute_fidelity_with_shadows(shadows, circ, 50, 80)

0.785

As you can see, we have received a good estimate of the expectation value.